## Merging relevance and click labels

In [44]:
import pandas as pd

relevance_df_1 = pd.read_excel("./data/QL_IT_oct_7_exp_on_results.xlsx")
relevance_df_2 = pd.read_excel("./data/QL_IT_oct_8_exp_on_results.xlsx")
relevance_df_3 = pd.read_excel("./data/QL_IT_oct_10_exp_on_results.xlsx")
rel_df_other = pd.read_excel("./data/QL_IT_exp_on_results.xlsx")
emotional_rel_df = pd.read_excel("./data/QL_IT_oct__exp_on_results.xlsx")
relevance_df = pd.concat([relevance_df_1, relevance_df_2, relevance_df_3, rel_df_other, emotional_rel_df])
# relevance_df = pd.concat([relevance_df_1, relevance_df_2, relevance_df_3])
relevance_df["user_query"] = [query.strip() for query in relevance_df["user_query"]] # removing leading/trailing white spaces in queries
relevance_df.drop(["target"], axis=1, inplace=True)
relevance_df.drop_duplicates(inplace=True)
relevance_df.to_excel("./data/search_results.xlsx", index=False)

clicked_df_1 = pd.read_excel("./data/QL_Oct 7_clicked.xlsx")
clicked_df_2 = pd.read_excel("./data/QL_Oct 8_clicked.xlsx")
clicked_df_3 = pd.read_excel("./data/QL_Oct 10_clicked.xlsx")
clicked_df = pd.concat([clicked_df_1, clicked_df_2, clicked_df_3])
clicked_df["Query"] = [query.strip() for query in clicked_df["Query"]] # removing leading/trailing white spaces in queries
clicked_df.drop_duplicates(inplace=True)
clicked_df.to_excel("./data/clicked_results.xlsx", index=False)

In [45]:
print(len(clicked_df_1["Query"].unique()), len(clicked_df_2["Query"].unique()), len(clicked_df_3["Query"].unique())) 
print(len(clicked_df["Query"].unique()))
print(len(relevance_df_1["user_query"].unique()), len(relevance_df_2["user_query"].unique()), len(relevance_df_3["user_query"].unique()), len(rel_df_other["user_query"].unique()), len(emotional_rel_df["user_query"].unique()))
print(len(relevance_df["user_query"].unique()))

40 16 18
70
45 22 20 211 82
282


In [48]:
relevance_df.head(1)

,session_id,user_query,url,source,title,snippet,position,TextSample,TextSample-Tokenized,tagged_TextSample-Tokenized,...,E_anticipation,E_disgust,E_fear,E_joy,E_sadness,E_surprise,E_trust,S_pos,S_neg,S_obj
0,393,le conseguenze negative della distruzione dell...,https://www.ecoage.it/conseguenze-della-defore...,R,Conseguenze della deforestazione - ecoage.it C...,Conseguenze della deforestazione . La deforest...,1,Conseguenze della deforestazione - ecoage.it C...,"['Conseguenze', 'della', 'deforestazione', '-'...","[('Conseguenze', 'NNP'), ('della', 'NN'), ('de...",...,0.0,0.2,0.2,0.0,0.2,0.0,0.0,0.0,0.25,0.75


In [49]:
clicked_df.head(1)

,Query,url,title,position
0,le conseguenze negative della distruzione dell...,http://www.eniscuola.net/argomento/foresta-tro...,Conseguenze della deforestazione - eniscuola.net,5


In [46]:
len(relevance_df), len(clicked_df)

(3683, 111)

In [52]:
def get_search_session_stats(relevance_df, df_name):
    session_stats = []

    for session_id in relevance_df["session_id"].unique():
        search_session = relevance_df.loc[relevance_df["session_id"]==session_id]
        for query in search_session["user_query"].unique():
            search_results = search_session.loc[search_session["user_query"]==query]
            session_stats.append([session_id, query, len(search_results), len(search_results["url"].unique())])

    df = pd.DataFrame(session_stats, columns=["session_id", "user_query", "num_results", "num_unique_urls"])
    df.to_excel("./data/search_session_stats_"+df_name+".xlsx", index=False)
    df

In [53]:
get_search_session_stats(relevance_df, df_name="relevance_df")

In [50]:
print(len(relevance_df["url"].unique()), len(relevance_df["TextSample"].unique()), len(relevance_df["title"].unique()), len(relevance_df["snippet"].unique()))

1556 1995 1612 1965


In [58]:
rows_to_remove = []

for session_id in relevance_df["session_id"].unique():
    search_session = relevance_df.loc[relevance_df["session_id"]==session_id] # search results for a session id
    for query in search_session["user_query"].unique():
        search_results = search_session.loc[search_session["user_query"]==query] # search results for a query within a session
        if len(search_results) != len(search_results["url"].unique()): # check if the same webpage appeared more than once in the SERP for query during a given session
            for url in search_results["url"].unique():
                if len(search_results.loc[search_results["url"]==url]) > 1:
                    rows_to_remove += list(search_results.loc[search_results["url"]==url].index)

print(rows_to_remove)
relevance_df_cleaned = relevance_df.drop(rows_to_remove)
relevance_df_cleaned.head(2)

[11, 21, 16, 26, 41, 42, 51, 61, 52, 53, 54, 64, 86, 87, 91, 101, 95, 97, 124, 125, 126, 128, 137, 138, 139, 142, 153, 154, 161, 162, 208, 209, 325, 326, 374, 375, 377, 378, 70, 71, 80, 81, 436, 440, 437, 439, 0, 444, 1, 445, 2, 446, 3, 447, 4, 448, 6, 450, 7, 451, 8, 452, 9, 453, 10, 454, 12, 456, 13, 457, 15, 459, 16, 460, 17, 461, 18, 462, 19, 463, 20, 464, 21, 465, 22, 466, 23, 467, 24, 468, 25, 469, 26, 470, 27, 471, 28, 472, 39, 475, 40, 476, 43, 479, 44, 480, 45, 481, 47, 483, 48, 484, 49, 485, 50, 486, 54, 490, 55, 491, 56, 492, 57, 493, 59, 495, 60, 67, 496, 503, 62, 498, 63, 499, 66, 502, 68, 504, 70, 506, 72, 508, 74, 510, 75, 511, 78, 79, 80, 516, 88, 89, 524, 525, 91, 527, 92, 528, 93, 529, 94, 530, 95, 531, 96, 532, 98, 100, 534, 536, 102, 538, 103, 539, 104, 540, 105, 541, 108, 544, 109, 545, 110, 546, 112, 548, 115, 551, 116, 552, 119, 122, 555, 558, 123, 559, 127, 563, 128, 564, 129, 565, 131, 567, 132, 568, 137, 573, 138, 574, 139, 575, 141, 577, 144, 580, 145, 581, 1

,session_id,user_query,url,source,title,snippet,position,TextSample,TextSample-Tokenized,tagged_TextSample-Tokenized,...,E_anticipation,E_disgust,E_fear,E_joy,E_sadness,E_surprise,E_trust,S_pos,S_neg,S_obj
14,395,Dove si trova l isola di plastica,https://it.blastingnews.com/ambiente/2018/03/v...,NR,L'isola di plastica nel mezzo dell'oceano Paci...,L'isola di plastica nel mezzo dell'oceano Paci...,5,L'isola di plastica nel mezzo dell'oceano Paci...,"[""L'isola"", 'di', 'plastica', 'nel', 'mezzo', ...","[(""L'isola"", 'NNP'), ('di', 'NN'), ('plastica'...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.307692,0.076923,0.615385
30,395,I ghiacciai in via di estinzione,https://trekking.it/news/ghiacciai-specie-in-e...,R,"Ghiacciai, specie in estinzione | Trekking.it",Un esempio per tutti: la Groenlandia rappresen...,1,"Ghiacciai, specie in estinzione | Trekking.it....","['Ghiacciai', ',', 'specie', 'in', 'estinzione...","[('Ghiacciai', 'NNP'), (',', ','), ('specie', ...",...,0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.117647,0.058824,0.823529


In [59]:
get_search_session_stats(relevance_df=relevance_df_cleaned, df_name="relevance_df_cleaned")

In [60]:
print(len(relevance_df_cleaned["session_id"].unique()), len(relevance_df["session_id"].unique()))

88 94


In [29]:
weird_ones = 0
weird_queries_count = 0
weird_queries = []
weird_titles = []

for query in clicked_df["Query"].unique():
    weird_query_flag = 0
    temp = clicked_df.loc[clicked_df["Query"]==query]
    for _, row in temp.iterrows():
        user_query = row["Query"]
        assert user_query==query
        clicked_url = row["url"]
        clicked_title = row["title"]
        clicked_position = row["position"]
        clicked_results = clicked_df.loc[(clicked_df["Query"]==user_query)&(clicked_df["url"]==clicked_url)&(clicked_df["title"]==clicked_title)&(clicked_df["position"]==clicked_position)]
        if len(clicked_results)>1:
            print(clicked_results)
            break
        found_result = relevance_df.loc[(relevance_df["user_query"]==user_query) & (relevance_df["title"] ==clicked_title) & (relevance_df["url"]==clicked_url) & (relevance_df["position"]==clicked_position)]
        if len(found_result) > 1:
            if len(found_result)!=len(found_result["session_id"].unique()):
                weird_query_flag = 1
            # print(found_result["session_id"].unique())
            # print(user_query)
            # print(clicked_title)
            # print(clicked_url)
            # print(clicked_position)
            # print(found_result["snippet"].unique())
            # print(found_result["TextSample"].unique())
            weird_ones += 1
            weird_titles.append(clicked_title)
    if weird_query_flag == 1:
        weird_queries_count += 1
        weird_queries.append(query)

print(weird_queries_count)
for query in weird_queries:
    print(query)

26
perchÃƒÂ© gli orsi polari si stanno estinguendo
Le regole per tenere le spiagge pulite
persona che rispetta la natura
Dove si trova Ã„Âº isola di plastica
Cosa ÃƒÂ¨ Ã„Â¾isola di plastica
Dove si trova Ã„Â¾isola di plastica?
Di cosa si occupa il WWF ?
Cosa ÃƒÂ¨  Ocean  Cleanup
Dove si trova Ã„Â¾isola di plastica Cosa ÃƒÂ¨
Cosa ÃƒÂ¨ ocean cleanup? Italino
Cosa ÃƒÂ¨ ocean cleanup? Ita
Qual ÃƒÂ¨ la causa della riduzione del lago Aral
Quali sono le regole per tenere le spiagge pulite ?
Quali sono le regole per tenere le spiagge pulite?
cos e la biodiversitÃƒÂ
Dove si trova Ã„Â¾ isola di plastica?cosa ÃƒÂ¨?
Cosa e la isola di plastica
perchÃƒÂ¨ gli orsi polari stanno diminuendo drasticamente di numero ?
PerchÃƒÂ© gli orsi polari stanno diminuendo drasticamente di numero?
Di cosa si occupa il wwf?
Di che cosa si occupa il wwf
A cosa serve frigorifero solidale
Cosa ÃƒÂ¨  Oceano cleanup
Qual ÃƒÂ© la causa della riduzione del lago Aral
Quale ghiacciaio si sta scioglendo in modo preoccupante
C

In [30]:
clicked_df.head()

,Query,url,title,position
0,le conseguenze negative della distruzione dell...,http://www.eniscuola.net/argomento/foresta-tro...,Conseguenze della deforestazione - eniscuola.net,5
1,BiodiversitÃƒÂ¡,http://www.globalissues.org/article/170/why-is...,Why Is Biodiversity Important? Who Cares? â€” ...,9
2,perchÃƒÂ© gli orsi polari si stanno estinguend...,http://www.ilcambiamento.it/articoli/orsi_pola...,"Orsi polari, Ã¨ allarme estinzione. Lo conferm...",5
3,Lisola di plastica cosa ÃƒÂ¨,http://www.lascimmiaviaggiatrice.it/isola-di-p...,isola di plastica che cos'Ã¨ e perchÃ¨ non puÃ...,3
4,Quanto psa lisola di plastica,http://www.meteoweb.eu/2018/03/inquinamento-oc...,Inquinamento: la piÃ¹ grande isola ... - meteo...,6


In [31]:
relevance_df.head(1)

,session_id,user_query,url,source,title,snippet,position,TextSample,TextSample-Tokenized,tagged_TextSample-Tokenized,...,E_anticipation,E_disgust,E_fear,E_joy,E_sadness,E_surprise,E_trust,S_pos,S_neg,S_obj
0,393,le conseguenze negative della distruzione dell...,https://www.ecoage.it/conseguenze-della-defore...,R,Conseguenze della deforestazione - ecoage.it C...,Conseguenze della deforestazione . La deforest...,1,Conseguenze della deforestazione - ecoage.it C...,"['Conseguenze', 'della', 'deforestazione', '-'...","[('Conseguenze', 'NNP'), ('della', 'NN'), ('de...",...,0.0,0.2,0.2,0.0,0.2,0.0,0.0,0.0,0.25,0.75


In [32]:
print(len(relevance_df["user_query"]), len(clicked_df["Query"]))
print(len(relevance_df["user_query"].unique()), len(clicked_df["Query"].unique()))
print(clicked_df["Query"].value_counts())

3683 111
282 70
Query
Di cosa si occupa il WWF ?                                         4
Isola di plastica                                                  4
Qual ÃƒÂ© la causa della riduzione del lago Aral                   3
Conseguenze negative della distruzione della foresta amazzonica    3
perchÃƒÂ© gli orsi polari si stanno estinguendo sito babbanas      3
                                                                  ..
Peso bottiglia di plastica                                         1
spiagge bandiera blu 2019                                          1
Spiagge con la bandiera della pulizia                              1
Spiagge con bandiera blu                                           1
CosÃƒÂ¨ Ã„Â¾isola di plastica                                      1
Name: count, Length: 70, dtype: int64


In [33]:
rel_df_queries = list(relevance_df["user_query"].unique())
click_df_queries = list(clicked_df["Query"].unique())

count = 0
for query in click_df_queries:
    if query not in rel_df_queries:
        print(query)
        count+=1

print("Number of queries with no relevance judgments: ", count)

Number of queries with no relevance judgments:  0


In [34]:
combined_data = []

for query in click_df_queries:
    search_results = relevance_df.loc[relevance_df["user_query"]==query] # all SE results yielded for a query
    clicked_results = clicked_df.loc[clicked_df["Query"]==query] # all results clicked for same query
    for _, row in search_results.iterrows():
        session_id = row["session_id"]
        user_query = row["user_query"]
        title = row["title"]
        url = row["url"]
        snippet = row["snippet"]
        textsample = row["TextSample"]
        textsample_tokens = row["TextSample-Tokenized"]
        textsample_pos = row["tagged_TextSample-Tokenized"]
        position = row["position"]
        relevant = row["source"]

        clicked_result_found = clicked_results.loc[(clicked_results["url"]==url)&(clicked_results["title"]==title)&(clicked_results["position"]==position)] # find the clicked result corresponding to (query+url+title+SERP position) combo
        assert len(clicked_result_found) <= 1 
        
        if len(clicked_result_found)==1:
            click = "C"
        elif len(clicked_result_found)==0:
            click="NC"
        
        combined_data.append([session_id, query,url,title,snippet,textsample,textsample_tokens, textsample_pos, position, relevant, click])


#     clicked_urls = list(clicked_df.loc[clicked_df["Query"]==query]["url"])
#     resources = relevance_df.loc[relevance_df["user_query"]==query]
#     for _, row in resources.iterrows():
#         session_id = row["session_id"]
#         title = row["title"]
#         snippet = row["snippet"]
#         textsample = row["TextSample"]
#         position = row["position"]
#         relevant = row["source"]
#         url = row["url"]
#         if url in clicked_urls:
#             click = "C"
#         else:
#             click = "NC"

#         combined_data.append([session_id, query,url,title,snippet,textsample, position, relevant, click])
    

combined_data_df = pd.DataFrame(combined_data, columns = ["session_id", "query", "url", "title", "snippet", "text_sample", "text_sample_tokens", "text_sample_pos", "position", "relevant", "click"])
# combined_data_df.drop(["position"], axis=1, inplace=True)
combined_data_df.drop_duplicates(inplace=True, ignore_index=True)
combined_data_df.to_excel("./data/query_log_IT.xlsx", index=False)
print(len(combined_data_df))
combined_data_df.head(2)

728


,session_id,query,url,title,snippet,text_sample,text_sample_tokens,text_sample_pos,position,relevant,click
0,393,le conseguenze negative della distruzione dell...,https://www.ecoage.it/conseguenze-della-defore...,Conseguenze della deforestazione - ecoage.it C...,Conseguenze della deforestazione . La deforest...,Conseguenze della deforestazione - ecoage.it C...,"['Conseguenze', 'della', 'deforestazione', '-'...","[('Conseguenze', 'NNP'), ('della', 'NN'), ('de...",1,R,NC
1,393,le conseguenze negative della distruzione dell...,https://forestaamazzonica.blogspot.com/2016/08...,La deforestazione in Amazzonia - Foresta Amazz...,Le loro stime sulla deforestazione sono deriva...,La deforestazione in Amazzonia - Foresta Amazz...,"['La', 'deforestazione', 'in', 'Amazzonia', '-...","[('La', 'NNP'), ('deforestazione', 'NN'), ('in...",2,NR,NC


In [35]:
len(combined_data_df["query"].unique())

70

In [36]:
combined_data_df["query"].value_counts()

query
perchÃƒÂ© gli orsi polari si stanno estinguendo                        22
PerchÃƒÂ¨ gli orsi polari stanno diminuendo drasticamente di numero    20
Spiagge con bandiera blu                                               18
Le regole per tenere le spiagge pulite                                 13
Anno con temperature piÃƒÂ¹ alte                                       12
                                                                       ..
Spiagge con bandiera della loro pulizia                                10
PerchÃƒÂ¨ gli orsi polari diminuiscono?                                10
cos e la biodiversitÃƒÂ                                                 9
perchÃƒÂ© gli orsi polari si stanno estinguendo sito babbanas           6
ocean cleanup                                                           5
Name: count, Length: 70, dtype: int64

In [37]:
combined_data_df.groupby(["click", "relevant"]).size()

click  relevant
C      NR            3
       R           112
NC     NR          603
       R            10
dtype: int64

In [38]:
combined_data_excluding_weird_queries = []

for query in click_df_queries:
    if query in weird_queries: # queries within the same session with multiple snippets corresponding to the same web title
        continue
    search_results = relevance_df.loc[relevance_df["user_query"]==query] # all SE results yielded for a query
    clicked_results = clicked_df.loc[clicked_df["Query"]==query] # all results clicked for a query
    for _, row in search_results.iterrows():
        session_id = row["session_id"]
        user_query = row["user_query"]
        title = row["title"]
        url = row["url"]
        snippet = row["snippet"]
        textsample = row["TextSample"]
        textsample_tokens = row["TextSample-Tokenized"]
        textsample_pos = row["tagged_TextSample-Tokenized"]
        position = row["position"]
        relevant = row["source"]

        clicked_result_found = clicked_results.loc[(clicked_results["url"]==url)&(clicked_results["title"]==title)&(clicked_results["position"]==position)] # find the clicked result corresponding to (query+url+title+SERP position) combo
        assert len(clicked_result_found) <= 1 
        
        if len(clicked_result_found)==1:
            click = "C"
        elif len(clicked_result_found)==0:
            click="NC"
        
        combined_data_excluding_weird_queries.append([session_id, query,url,title,snippet,textsample,textsample_tokens, textsample_pos, position, relevant, click])


#     clicked_urls = list(clicked_df.loc[clicked_df["Query"]==query]["url"])
#     resources = relevance_df.loc[relevance_df["user_query"]==query]
#     for _, row in resources.iterrows():
#         session_id = row["session_id"]
#         title = row["title"]
#         snippet = row["snippet"]
#         textsample = row["TextSample"]
#         position = row["position"]
#         relevant = row["source"]
#         url = row["url"]
#         if url in clicked_urls:
#             click = "C"
#         else:
#             click = "NC"

#         combined_data.append([session_id, query,url,title,snippet,textsample, position, relevant, click])
    

combined_data_excluding_weird_queries_df = pd.DataFrame(combined_data_excluding_weird_queries, columns = ["session_id", "query", "url", "title", "snippet", "text_sample", "text_sample_tokens", "text_sample_pos", "position", "relevant", "click"])
# combined_data_df.drop(["position"], axis=1, inplace=True)
combined_data_excluding_weird_queries_df.drop_duplicates(inplace=True, ignore_index=True)
combined_data_excluding_weird_queries_df.to_excel("./data/query_log_IT_excluding_weird_queries.xlsx", index=False)
print(len(combined_data_excluding_weird_queries_df))
combined_data_excluding_weird_queries_df.head(2)

453


,session_id,query,url,title,snippet,text_sample,text_sample_tokens,text_sample_pos,position,relevant,click
0,393,le conseguenze negative della distruzione dell...,https://www.ecoage.it/conseguenze-della-defore...,Conseguenze della deforestazione - ecoage.it C...,Conseguenze della deforestazione . La deforest...,Conseguenze della deforestazione - ecoage.it C...,"['Conseguenze', 'della', 'deforestazione', '-'...","[('Conseguenze', 'NNP'), ('della', 'NN'), ('de...",1,R,NC
1,393,le conseguenze negative della distruzione dell...,https://forestaamazzonica.blogspot.com/2016/08...,La deforestazione in Amazzonia - Foresta Amazz...,Le loro stime sulla deforestazione sono deriva...,La deforestazione in Amazzonia - Foresta Amazz...,"['La', 'deforestazione', 'in', 'Amazzonia', '-...","[('La', 'NNP'), ('deforestazione', 'NN'), ('in...",2,NR,NC


In [39]:
len(combined_data_excluding_weird_queries_df["query"].unique())

44

In [40]:
combined_data_excluding_weird_queries_df["query"].value_counts()

query
PerchÃƒÂ¨ gli orsi polari stanno diminuendo drasticamente di numero      20
Spiagge con bandiera blu                                                 18
Anno con temperature piÃƒÂ¹ alte                                         12
Dove si trova l isola di plastica                                        12
le conseguenze negative della distruzione della foresta amazzonica       10
spiagge bandiera blu 2019                                                10
CosÃƒÂ© il buco nel azono                                                10
Nomi spiagge bandiera pulizia                                            10
Temperature estive superano quelle misurate nel 2019                     10
Regole per tenere pulite le spiagge                                      10
Le spiagge piÃ…Â« piÃƒÂ¹ pulite                                          10
Peso bottiglia di plastica                                               10
Spiagge con la bandiera della pulizia                                    10
Cose i

In [41]:
combined_data_excluding_weird_queries_df.groupby(["click", "relevant"]).size()

click  relevant
C      R            66
NC     NR          380
       R             7
dtype: int64

## Misc

In [216]:
count_clicked_urls_missed = 0

for _, row in combined_data_df.iterrows():
    result_url = row["url"]
    click_label = row["click"]
    if click_label == "C":
        if len(clicked_df.loc[clicked_df["url"]==result_url])==0:
            print(row)
    if row["click"]=="NC":
        if len(clicked_df.loc[clicked_df["url"]==result_url])!=0:
            print(result_url, row["session_id"])
            print(list(combined_data_df.loc[combined_data_df["url"]==result_url]["session_id"]))
#             count_clicked_urls_missed+=1

# print(count_clicked_urls_missed)

https://www.ecoage.it/conseguenze-della-deforestazione.htm 393
[393, 417]
https://forestaamazzonica.blogspot.com/2016/08/deforestazione-amazzonia.html 393
[393, 417, 417]
https://it.wikipedia.org/wiki/Pacific_Trash_Vortex 401
[401, 398, 395, 404, 404, 401, 401, 499]
http://www.saperescienza.it/rubriche/geologia/l-isola-di-plastica-18-07-2014/200-l-isola-di-plastica-18-07-2014 401
[401, 395, 395, 404, 401, 401]
http://www.ilbullone.org/lisola-di-plastica/ 401
[401, 395, 499, 499]
https://it.wikipedia.org/wiki/Pacific_Trash_Vortex 398
[401, 398, 395, 404, 404, 401, 401, 499]
http://www.rainews.it/dl/rainews/articoli/russia-orsi-polari-14b21631-5bff-48f1-93c1-749824c1c39c.html 481
[481]
http://www.ilbullone.org/lisola-di-plastica/ 395
[401, 395, 499, 499]
https://gosblogsurf.blogspot.com/2011/01/lisola-di-plastica.html 395
[395, 499]
https://it.answers.yahoo.com/question/index?qid=20090218083835AAVMgRL 411
[411, 435]
https://www.elledecor.com/it/lifestyle/a28066697/incendi-foresta-amazzon

In [60]:
import pandas as pd

df = pd.read_excel("./data/query_log_IT.xlsx")
df.head()

,query,url,title,snippet,text_sample,position,relevant,click
0,le conseguenze negative della distruzione dell...,https://www.ecoage.it/conseguenze-della-defore...,Conseguenze della deforestazione - ecoage.it C...,Conseguenze della deforestazione . La deforest...,Conseguenze della deforestazione - ecoage.it C...,1,R,NC
1,le conseguenze negative della distruzione dell...,https://forestaamazzonica.blogspot.com/2016/08...,La deforestazione in Amazzonia - Foresta Amazz...,Le loro stime sulla deforestazione sono deriva...,La deforestazione in Amazzonia - Foresta Amazz...,2,NR,NC
2,le conseguenze negative della distruzione dell...,https://www.elledecor.com/it/lifestyle/a280666...,Le conseguenze degli incendi della foresta ama...,Il Brasile sta vivendo la piÃ¹ grande ondata d...,Le conseguenze degli incendi della foresta ama...,3,R,C
3,le conseguenze negative della distruzione dell...,https://www.informazioneambiente.it/deforestaz...,Cause e conseguenze della deforestazione ambie...,Le conseguenze della deforestazione. Ma a cosa...,Cause e conseguenze della deforestazione ambie...,4,NR,NC
4,le conseguenze negative della distruzione dell...,http://www.eniscuola.net/argomento/foresta-tro...,Conseguenze della deforestazione - eniscuola.net,Conseguenze della deforestazione. Le foreste h...,Conseguenze della deforestazione - eniscuola.n...,5,R,C


In [62]:
len(df["query"].unique()), len(df["text_sample"].unique()), len(df)

(70, 536, 711)

In [ ]:
import pandas as pd

